## Naivni Bayesov klasifikator

## Naive Bayes Classifier

In [1]:
import numpy as np
import pandas as pd

### Primer za ogrevanje

V letniku športne gimnazije imamo 20 učencev. Vsak od njih sodeluje pri enem od športov: ```kosarka```, ```nogomet```, ```gimnastika```. Njihovo višino smo ocenili "na oko" in vsakemu učencu pripisali eno od možnih vrednosti: ```nizek```, ```srednji``` ali ```visok```.

### Warmup example

There are 20 pupils in a year of sports high school. Each of them participates in one of the sports: ```basketball```, ```football```, ```gymnastics```. We evaluated their height "on the eye" and assigned to each pupil one of the possible values: ```low```, ```average``` or ```high```.

<img src="../slike/footballers.png" width=600/>

<font color="blue"> Kako bi novemu učencu Marku, ki je ```srednje``` rasti predlagali najprimernejši šport? </font>

<font color="blue">How would you suggest the most appropriate sport for the new pupil Mark, who is of ```average``` height? </font>

In [ ]:
data = pd.read_table('../data/sportniki.tab', skiprows=[1,2])

In [3]:
data.columns

Index(['visina', 'sport'], dtype='object')

In [4]:
data[:5]

,visina,sport
0,visok,kosarka
1,visok,kosarka
2,visok,kosarka
3,visok,kosarka
4,srednji,kosarka


Za začetek poglejmo kako popularni so posamezni športi:

To start, let's look at how popular each sport is:

In [5]:
for sport in pd.unique(data['sport']):
    subset = data.loc[data['sport'] == sport]
    
    print(sport)
    print(subset)
    print()
    
    py     = len(subset) / len(data)
    print("Sport (Y): %s, število: %d, verjetnost P(Y): %f" % (sport, len(subset), py))

kosarka
    visina    sport
0    visok  kosarka
1    visok  kosarka
2    visok  kosarka
3    visok  kosarka
4  srednji  kosarka
5  srednji  kosarka
6    nizek  kosarka
7    visok  kosarka

Sport (Y): kosarka, število: 8, verjetnost P(Y): 0.400000
nogomet
     visina    sport
8   srednji  nogomet
9   srednji  nogomet
10  srednji  nogomet
11    visok  nogomet
12    visok  nogomet
13    nizek  nogomet
14    nizek  nogomet

Sport (Y): nogomet, število: 7, verjetnost P(Y): 0.350000
gimnastika
     visina       sport
15    nizek  gimnastika
16    nizek  gimnastika
17    nizek  gimnastika
18  srednji  gimnastika
19  srednji  gimnastika

Sport (Y): gimnastika, število: 5, verjetnost P(Y): 0.250000


Najpopularnejši šport je košarka, s katerim se ukvarja 8 oz. 40% učencev. Naš prvi predlog je torej, naj se Marko ukvarja s košarko. S tem rezultatom nismo najbolj zadovoljni, saj vidimo da med košarkaši ni veliko športnikov ```srednje``` višine. Razlog? Pri izračunu nismo upoštevali verjetnosti lastnosti oz. *atributa* o Markovi višini.

<div style="background-color:#00ccff; margin-left:50px; margin-right:50px"> Splošnim verjetnostmi razredov, ki smo jih izračunali pravimo *apriorne* verjetnosti.

Označimo jih s $P(Y)$, kjer je $Y$ spremenljivka razreda.
</div>

V našem primeru $Y$ zavzame vrednostmi {```kosarka```, ```nogomet```, ```gimnastika```}.

The most popular sport is basketball, with 8 or 40% of pupils participating in it. Our first suggestion is that Marko should play basketball. This result is not the most satisfying, as we see that among basketball players there are not many athletes of ```medium``` height. The reason? In calculating, we did not take into account the probability of the property or *attribute* about Mark's height.

<div style="background-color:#00ccff; margin-left:50px; margin-right:50px"> The general probabilities of the classes that we calculated are called *a priori* probabilities.

Let us label them with $P(Y)$, where $Y$ is a class variable.
</div>

In our example, $Y$ takes on the values {```basketball```, ```football```, ```gymnastics```}.

In [6]:
for sport in pd.unique(data['sport']):
    subset_y = data.loc[data['sport'] == sport]
    subset_x = subset_y.loc[data['visina'] == 'srednji']
    p_xy = len(subset_x) / len(subset_y)
    
    print("Sport (Y): %s, št. srednje visokih: %d, verjetnost P(X=srednji|Y=%s): %f" % (sport, len(subset_x), sport, p_xy, ))
    print(subset_x)
    print()

Sport (Y): kosarka, št. srednje visokih: 2, verjetnost P(X=srednji|Y=kosarka): 0.250000
    visina    sport
4  srednji  kosarka
5  srednji  kosarka

Sport (Y): nogomet, št. srednje visokih: 3, verjetnost P(X=srednji|Y=nogomet): 0.428571
     visina    sport
8   srednji  nogomet
9   srednji  nogomet
10  srednji  nogomet

Sport (Y): gimnastika, št. srednje visokih: 2, verjetnost P(X=srednji|Y=gimnastika): 0.400000
     visina       sport
18  srednji  gimnastika
19  srednji  gimnastika




<br/>
Zanimivo! Verjetnost ```srednje``` višine je največja med nogometaši. Ali podatek zadošča za spremembo prvotne odločitve?

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
Verjetnosti $P(X|Y)$ pravimo <i>pogojna verjetnost spremenljivke $X$ pri znanem $Y$</i>.  Opredeljuje verjetnost, da je v primerih razreda $Y$ atribut $X$ zavzame določeno vrednost. 
</div>

Katera verjetnost pa nas v resnici zanima? Želimo, da izračun upošteva Markovo višino in oceni verjetnost vsakega od športov. To je verjetnost

$$ P(Y|X) $$

oz. v Markovem primeru

$$ P(Y|X=srednji)$$

Za izračun te verjetnosti uporabimo

<br/>
Interesting! The likelihood of a `` medium '' height is the highest among footballers. Is the information sufficient to change the original decision?

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
The probabilities of $P(X|Y)$ are called <i>pogojna verjetnost spremenljivke $X$ pri znanem $Y$</i>. It determines the probability that in the cases of the $Y$ class the attribute $X$ takes a certain value.
</div>

What probability does we really care about? We want the calculation to take Mark's height into account and evaluate the likelihood of each of the sports. That's the probability

$$ P(Y|X) $$

or. in Mark's case

$$ P(Y|X=srednji)$$

We use this probability to calculate this probability

## Bayesov obrazec

Da bi izračunali verjetno razreda pri danih atributih $P(Y|X)$, potrebujemo verjetnost za vse možne kombinacije razreda $Y$ in atributov $X$, ki jo označimo z $P(X, Y)$. Iz pravil o pogojni verjetnosti sledi:

$$ P(X, Y) = P(X|Y) \cdot P(Y) = P(Y|X) \cdot P(X)$$ 

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
Iz česar sledi <i>Bayesov obrazec</i> za izračun $P(Y|X)$:

$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)} $$ 
</div>
<br/>

Izračun verjetnosti razreda $Y$ pri znanih atributih $X$ je torej odvisen od apriorne verjetnosti razreda $P(Y)$, pogojne verjetnosti $P(X|Y)$ in apriorne verjetnosti atributov $P(X)$. <font color="blue">V Markovem primeru torej:</font>

$$P(Y|X=srednji) = \frac{P(X=srednji|Y) \cdot P(Y)}{P(X=srednji)} $$ 


<br/>
<br/>
Če verjetnost ocenimo za vsako možno vrednost razreda Y, torej {```kosarka```, ```nogomet```, ```gimnastika```}, dobimo odgovor na prvotno vprašanje.

## Bayes form

In order to calculate the likely class for given attributes of $P(Y|X)$, we need probability for all possible combinations of the class $Y$ and attributes $X$, which is denoted by $P(X, Y)$. It follows from the rules on conditional probability:

$$ P(X, Y) = P(X|Y) \cdot P(Y) = P(Y|X) \cdot P(X)$$

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
What follows is <i>Bayesov obrazec</i> for calculating $P(Y|X)$:

$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)} $$
</div>
<br/>

The calculation of the probability of the class $Y$ in the known attribute of $X$ is therefore dependent on the a priori probability of the class $P(Y)$, the conditional probability of $P(X|Y)$, and the a priori probability of the attribute of $P(X)$. <font color="blue">V In the example of Mark, then: </font>

$$P(Y|X=srednji) = \frac{P(X=srednji|Y) \cdot P(Y)}{P(X=srednji)} $$


<br/>
<br/>
If we estimate the probability for each possible value of the class Y, then {`` `basketball``,` `football```,` `` gymnastics```, we get the answer to the original question.

In [7]:
for sport in pd.unique(data['sport']):
    
    subset_y  = data.loc[data['sport'] == sport]        # vsi sportniki danega sporta
    subset_x  = data.loc[data['visina'] == 'srednji']     # vsi srednje visoki ucenci
    
    subset_xy = subset_y.loc[data['visina'] == 'srednji'] # vsi srednje visoki ucenci v danem sportu
    
    # Izracunamo verjetnosti
    p_y  = len(subset_y)  / len(data)         
    p_x  = len(subset_x)  / len(data)
    p_xy = len(subset_xy) / len(subset_y)
    
    p_yx = (p_xy * p_y) / p_x
    
    print("Sport (Y): %s, napoved P(Y=%s | X=srednji): %f" % (sport, sport, p_yx))

Sport (Y): kosarka, napoved P(Y=kosarka | X=srednji): 0.285714
Sport (Y): nogomet, napoved P(Y=nogomet | X=srednji): 0.428571
Sport (Y): gimnastika, napoved P(Y=gimnastika | X=srednji): 0.285714


## Implementacija Naivnega Bayesovega klasifikatorja

## Implementation of the Naive Bayes Classifier

*Naivni Bayesov klasifikator* predpostavlja, da so atributi neodvisni med seboj, pri znanem razredu.

$$ P(Y|X_1, X_2, ..., X_p) = \frac{P(Y) \cdot P(X_1|Y) \cdot P(X_2|Y) \cdots P(X_p|Y)}{P(X)} $$

The *Naive Bayes classifier* assumes that the attributes are independent of each other, with known class.

$$ P(Y|X_1, X_2, ..., X_p) = \frac{P(Y) \cdot P(X_1|Y) \cdot P(X_2|Y) \cdots P(X_p|Y)}{P(X)} $$

##### Vprašanje 5-2-1
Dopolni implementacijo naivnega Bayesovega klasifikatorja, ki je definiran v spodnjem odseku. Dopolniti je potrebno del kode, kjer izračunamo 
* verjetnostne porazdelitev razredov $P(Y)$
* verjetnostne porazdelitve atributov pri znanem razredu $P(X|Y)$



### Sklepanje o podatkih

V primeru diskretnih atributov lahko obe porazdelitvi dobimo s *preštevanjem*.
* $P(Y)$ *Kolikokrat se v podatkih pojavi razred $Y$?*
* $P(X|Y)$ *Kolikokrat se v podatkih, ki spadajo v razred $Y$, pojavi atribut $X$?*


<font color="blue"><b>Kaj pa $P(X)$?</b></font> Ta verjetnost je včasih težko izračunljiva, posebej pri visoko dimenzionalnih podatkih, saj ni nujno, da bodo v podatki prisotne vse kombinacije atributov. Na srečo ta vrednost ne vpliva na izbiro najverjetnejšega razreda za posamezen primer!

### Napovedovanje

Za nov primer $X^* = (X_1^*, X_2^*, ..., X_p^*)$ med vsemi vrednostmi razreda $Y=y$, izberi tisto, ki maksimizira naslednji izraz:


$$ \text{arg max}_y \ P(Y=y) \cdot P(X_1^*|Y=y) \cdot P(X_2^*|Y=y) \cdots P(X_p^*|Y=y) $$

### Log-transformacija

Težava pri zgornjem pristopu je praktične narave; množenje velikega števila verjetnosti hitro privede do zelo majhnih števil, ki lahko presežejo strojno natančnost. Najenostavnejša rešitev, ki privede do enake izbire razreda je naslednja 

$$ \text{arg max}_y \ \text{log } P(Y=y) + \text{log } P(X_1|Y=y) + \text{log } P(X_2|Y=y) + ... + \text{log } P(X_p|Y=y) $$

##### Vprašanje 5-2-1
Complete the implementation of the Naive Bayes classifier, which is defined in the lower section. It is necessary to complete the part of the code where we calculate
* probability distribution of classes $P(Y)$
* probability distribution of attributes in the known class $P(X|Y)$

### Conclusion on data

In the case of discrete attributes, both distributions can be obtained by *counting*.
* $P(Y)$ *How many times does the $Y$ class appear in the data?*
* $P(X|Y)$ *How many times does the $X$ attribute appear in the data that belong to the $Y$ class?*

<font color="blue"><b>What about $P(X)$?</b></font> This probability is sometimes difficult to calculate, especially for high-dimensional data, since it is not necessary that all combinations of attributes will be present in the data. Fortunately, this value does not affect the choice of the most likely grade for a particular case!

### Predicting

For a new example  $X^* = (X_1^*, X_2^*, ..., X_p^*)$ among all values of the $Y=y$ class, select one that maximizes the following expression:

$$ \text{arg max}_y \ P(Y=y) \cdot P(X_1^*|Y=y) \cdot P(X_2^*|Y=y) \cdots P(X_p^*|Y=y) $$

### Log-transformation

The problem with the above approach is rather practical; multiplying a large number of probabilities quickly leads to very small numbers that can exceed machine accuracy. The simplest solution that leads to the same class choice is the following

$$ \text{arg max}_y \ \text{log } P(Y=y) + \text{log } P(X_1|Y=y) + \text{log } P(X_2|Y=y) + ... + \text{log } P(X_p|Y=y) $$

Pri implementaciji si pomagaj s podatki potnikov ladje <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

Podatke so že razdeljeni na učno in testno množico.

Naložimo učne podatke in izračunamo verjetnosti.

For the implementation help yourself with the passenger data from <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

The data is already divided into a learning and test set.

Load the learning data and calculate probabilities.

In [8]:
#data = Table('podatki/titanic-training.tab')
data = pd.read_table('podatki/titanic-training.tab', skiprows=[1,2])
print(data.columns[-1])
print(pd.unique(data['survived']))

# P(X=child | Y = yes)
survived_child  = data.loc[(data['age'] == 'child') & (data['survived'] == 'yes')]  
all_survived  = data.loc[data['survived'] == 'yes']

p_xy = len(survived_child) / len(all_survived)
p_xy

survived
['no' 'yes']


0.08379888268156424

In [9]:
data.columns[:-1]

Index(['status', 'age', 'sex'], dtype='object')

In [10]:
class NaiveBayes:
    """
    Naive Bayes classifier.
    
    :attribute self.probabilities
        Dictionary that stores
            - prior class probabilities P(Y)
            - attribute probabilities conditional on class P(X|Y)
    
    :attribute self.class_values
        All possible values of the class.
        
    :attribute self.variables
        Variables in the data. 
    
    :attribute self.trained
        Set to True after fit is called.
    """
    
    def __init__(self):
        self.trained       = False
        self.probabilities = dict()   
    
    
    def fit(self, data):
        """
        Fit a NaiveBayes classifier.
        
        :param data
            Orange data Table.        
        """
        class_variable      = data.domain.class_var    # class variable (Y) 
        self.class_values   = class_variable.values    # possible class values
        self.variables      = data.domain.attributes    # all other variables (X)
        
        n = len(data) # number of all data points
        
        # Compute P(Y)
        for y in self.class_values:

            # A not too smart guess (INCORRECT)
            self.probabilities[y] = 1/len(self.class_values)
            
            # <your code here>
            # Compute class probabilities and correctly fill
            #   probabilities[y] = ... 
            # Select all examples (rows) with class = y
          
            # </your code here>
        
        # Compute P(X|Y)
        for y in self.class_values:
            
            # Select all examples (rows) with class = y
            filty = SameValue(class_variable, y)
            
            for variable in self.variables:
                for x in variable.values:
                    
                    # A not too smart guess (INCORRECT)
                    p = 1 / (len(self.variables) * len(variable.values) * len(self.class_values))
                    
                    # P(variable=x|Y=y)
                    self.probabilities[variable, x, y] = p
                    
                
                    # <your code here>
                    # Compute correct conditional class probability
                    #   probabilities[x, value, c] = ... 
                    # 
                    # Select all examples with class == y AND 
                    # variable x == value
                    # Hint: use SameValue filter twice
            
                
                    # </your code here>
    
        self.trained = True
        
    
    def predict_instance(self, row):
        """
        Predict a class value for one row.
        
        :param row
            Orange data Instance.
        :return 
            Class prediction.
        """
        curr_p = float("-inf")   # Current highest "probability" (unnormalized)
        curr_c = None            # Current most probable class
        
        for y in self.class_values:
            p = np.log(self.probabilities[y])
            for x in self.variables:
                p = p + np.log(self.probabilities[x, row[x].value, y])
            
            if p > curr_p:
                curr_p = p
                curr_c = y
                
        return curr_c, curr_p
        
   

    def predict(self, data):
        """
        Predict class labels for all rows in data.
        
        :param data
            Orange data Table.       
        :return y
            NumPy vector with predicted classes.
        """
        
        n = len(data)
        predictions = list()
        confidences = np.zeros((n, ))
        
        for i, row in enumerate(data):
            pred, cf = self.predict_instance(row)
            predictions.append(pred)
            confidences[i] = cf
    
        return predictions, confidences

Rešitev je dostopna na: 205-2.ipynb

In [11]:
%run 205-2.ipynb

##  Uporaba klasifikatorja

## Using a classifier

Primer uporabe na podatkih potnikov ladje <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

An example of use on passenger data from <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

In [12]:
model = NaiveBayes()
model.fit(data)
model.probabilities

{'no': 0.6745454545454546,
 'yes': 0.32545454545454544,
 ('status', 'third', 'no'): 0.3450134770889488,
 ('status', 'second', 'no'): 0.12398921832884097,
 ('status', 'crew', 'no'): 0.4568733153638814,
 ('status', 'first', 'no'): 0.07412398921832884,
 ('age', 'adult', 'no'): 0.9663072776280324,
 ('age', 'child', 'no'): 0.03369272237196765,
 ('sex', 'male', 'no'): 0.9110512129380054,
 ('sex', 'female', 'no'): 0.0889487870619946,
 ('status', 'third', 'yes'): 0.24581005586592178,
 ('status', 'second', 'yes'): 0.17039106145251395,
 ('status', 'crew', 'yes'): 0.29329608938547486,
 ('status', 'first', 'yes'): 0.2905027932960894,
 ('age', 'adult', 'yes'): 0.9162011173184358,
 ('age', 'child', 'yes'): 0.08379888268156424,
 ('sex', 'male', 'yes'): 0.48044692737430167,
 ('sex', 'female', 'yes'): 0.5195530726256983}

In [13]:
predictions, confidences = model.predict(data)

for i in range(10):
    row = data.iloc[i]
    p = predictions[i]
    c = confidences[i]
    print("Status=%s, age=%s sex=%s" % (row['status'], row['age'], row['sex']))
    print("Actual class=%s, predicted class=%s confidence=%.5f" % (row['survived'], p, c))

Status=third, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.58532
Status=second, age=adult sex=female
Actual class=no, predicted class=yes confidence=-3.63450
Status=crew, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.30449
Status=crew, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.30449
Status=third, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.58532
Status=second, age=adult sex=male
Actual class=no, predicted class=no confidence=-2.60871
Status=crew, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.30449
Status=second, age=adult sex=male
Actual class=no, predicted class=no confidence=-2.60871
Status=third, age=adult sex=male
Actual class=yes, predicted class=no confidence=-1.58532
Status=third, age=adult sex=male
Actual class=no, predicted class=no confidence=-1.58532


## Ocenjevanje uspešnosti klasifikacije

Za ocenjevanje uspešnosti klasifikacije vsak napovedani primer primerjamo s pripadajočim resničnim razredom. Štirje možni izidi primerjave so naslednji: 

## Assessing the performance of the classification

In order to evaluate the success of the classification, we compare each predicted example with the corresponding real class. The four possible outcomes of the comparison are as follows:

<table>
<tr>
<td>
<ul>
<li>TP: True positives (pravilno napovedani pozitivni primeri)</li>
<li>FP: False positives (napačno napovedani negativni primeri)</li>
<li>TN: True negatives (pravilno napovedani negativni primeri)</li>
<li>FN: False negatives (napačno napovedani pozitini primeri)</li>
</ul> 

<br/>
<img src="slike/type12_error.jpeg" width=400/>

</td>
<td><img width="400" src="slike/Precisionrecall.png"></img><td>
<tr/>
<table>

<table>
<tr>
<td>
<ul>
<li>TP: True Positives (correctly predicted positive examples) </li>
<li>FP: False positives (wrongly predicted negative examples) </li>
<li>TN: True Negatives (correctly predicted negative examples) </li>
<li>FN: False negatives (wrongly predicted positive examples) </li>
</ul>

<br/>
<img src="slike/type12_error.jpeg" width=400/>

</td>
<td><img width="400" src="slike/Precisionrecall.png"></img><td>
<tr/>
<table>

### Delež pravilno razvrščenih razredov (ang. classification accuracy)

$$ca = \frac{TP + TN}{TP + TN + FP + FN}$$

<font color="green">Prednosti</font>:
* Enostaven izračun, jasna interpretacija
* Uporabna mera za poljubno število razredov

<font color="red">Slabosti</font>:
* Lahko zavaja pri neuravnoteženih porazdelitvah razredov

<br/>

### Ratio of correctly classified classes (classification accuracy)

$$ca = \frac{TP + TN}{TP + TN + FP + FN}$$

<font color="green">Pros</font>:
* Simple calculation, clear interpretation
* Useful measure for any number of classes

<font color="red">Cons</font>:
* It can be misleading with unbalanced class distributions

### Natančnost, priklic (ang. precision, recall)

$$ p = \frac{TP}{TP + FP} $$

$$ r = \frac{TP}{TP + FN} $$

<font color="green">Prednosti</font>:
* Enostaven izračun, jasna interpretacija
* Ločitev obeh tipov napak (napačno pozitivni in napačno negativni primeri)
* Uporabna tudi pri neuravnoteženih porazdelitvah razredov

<font color="red">Slabosti</font>:
* Uporabno pretežno za klasifikacijo v dva razreda
* Težko povzeti obe meri ; približek je F1-vrednost (ang. F1-score)
$$ F1 = 2 \frac{p \cdot r}{p + r} $$

### Precision, recall

$$ p = \frac{TP}{TP + FP} $$

$$ r = \frac{TP}{TP + FN} $$

<font color="green">Pros</font>:
* Simple calculation, clear interpretation
* Separation of both types of errors (incorrectly positive and wrongly negative examples)
* Also applicable for unbalanced classroom distributions

<font color="red">Cons</font>:
* Applicable predominantly for classification in two classes
* It is difficult to summarize both measures; the approximation is F1-value (F1-score)
$$ F1 = 2 \frac{p \cdot r}{p + r} $$

<font color="green"><b>Naredi sam/a.</b></font> Napovej razrede na testni množici. Napovedane razrede primerjaj z resničnimi in izmeri klasifikacijsko točnost, natančnost, priklic in F1-vrednost.

<font color="green"><b>Do it yourself.</b></font> Predict the classes on the test set. Compare the predicted classes with the real ones and measure the classification accuracy, precision, recall and F1 value.

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# uporaba metod: 
test_data = pd.read_table('podatki/titanic-test.tab', skiprows=[1,2])
predictions, _ = model.predict(test_data) 
truth          = [test_data.loc[i, "survived"] for i in range(len(test_data))]
accuracy_score(truth, predictions)

0.771117166212534

<font color="orange"><b>Izziv.</b></font> Nekateri atributi imajo verjetnost 0 pri posameznem razredu. Kako bi popravili klasifikator?

<font color="orange"><b>Challenge.</b></font> Some attributes have a probability of 0 for each class. How would you repair the classifier?

<font color="blue"><b>Razmisli.</b></font> Kako bi dopolnili klasifikator, če bi bili nekateri atributi lahko tudi zvezni? Namig: spomni se vaj, ko smo spoznali *verjetnostne porazdelitve* zveznih spremenljivk. 

<font color="blue"><b>Think.</b></font> How to complete a classifier if some of the attributes could also be continuous? Hint: remember the exercises when we learned about the probability distributions of continuous variables.